In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import seaborn as sns
import sklearn
import imblearn
import matplotlib.pyplot as plt
import time
import sklearn.metrics as m
import xgboost as xgb
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')


In [2]:
df = pd.read_csv('../datasets/target_model/CICIDS2017/df_target_0311.csv')

### Select Feature & modify special word/number

In [3]:
df = df.drop([' Destination Port', ' Fwd Packet Length Max', ' Fwd Packet Length Mean', ' Bwd Packet Length Min', ' Flow IAT Mean', 
              ' Flow IAT Max', ' Flow IAT Min',' Fwd IAT Std', ' Fwd IAT Min', 'Bwd IAT Total', 
              'Fwd PSH Flags' , ' Fwd URG Flags', ' Bwd URG Flags', ' Min Packet Length', ' Packet Length Mean', 
              ' Packet Length Variance', ' RST Flag Count', ' PSH Flag Count', ' URG Flag Count', 
              ' CWE Flag Count', ' ECE Flag Count', ' Average Packet Size', ' Avg Fwd Segment Size', ' CWE Flag Count', ' CWE Flag Count',
              'Fwd Avg Bytes/Bulk', ' Fwd Avg Packets/Bulk', ' Fwd Avg Bulk Rate', ' Bwd Avg Bytes/Bulk', ' Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate', 
              'Init_Win_bytes_forward', ' min_seg_size_forward', 'Active Mean', ' Active Min', 'Idle Mean', ' Idle Max', ' Idle Min'], axis=1)
df 

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Std,...,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_backward,act_data_pkt_fwd,Active Std,Active Max,Idle Std,Label,Binary_class,attack_type
0,5435785,4,0,24,0,6,0.000000,0,0.0,0.000000,...,0,0,-1,3,0.0000,952,0.000000,DDoS,malicious,DOS
1,26,1,1,2,6,2,0.000000,6,6.0,0.000000,...,1,6,0,0,0.0000,0,0.000000,PortScan,malicious,PortScan
2,23475,1,1,70,152,70,0.000000,152,152.0,0.000000,...,1,152,-1,0,0.0000,0,0.000000,BENIGN,benign,benign
3,37553,2,0,0,0,0,0.000000,0,0.0,0.000000,...,0,0,-1,0,0.0000,0,0.000000,BENIGN,benign,benign
4,71220,4,4,172,384,43,0.000000,96,96.0,0.000000,...,4,384,-1,3,0.0000,0,0.000000,BENIGN,benign,benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1261172,67,1,1,2,6,2,0.000000,6,6.0,0.000000,...,1,6,0,0,0.0000,0,0.000000,PortScan,malicious,PortScan
1261173,53972647,10,10,432,2052,0,136.610395,2052,205.2,648.899376,...,10,2052,939,1,156701.8442,387543,2062.257986,BENIGN,benign,benign
1261174,30836,2,2,86,200,43,0.000000,100,100.0,0.000000,...,2,200,-1,1,0.0000,0,0.000000,BENIGN,benign,benign
1261175,60317,1,1,51,67,51,0.000000,67,67.0,0.000000,...,1,67,-1,0,0.0000,0,0.000000,BENIGN,benign,benign


In [4]:
# Removing whitespaces in column names.
col_names = [col.replace(' ', '') for col in df.columns]
df.columns = col_names
df.head()

,FlowDuration,TotalFwdPackets,TotalBackwardPackets,TotalLengthofFwdPackets,TotalLengthofBwdPackets,FwdPacketLengthMin,FwdPacketLengthStd,BwdPacketLengthMax,BwdPacketLengthMean,BwdPacketLengthStd,...,SubflowBwdPackets,SubflowBwdBytes,Init_Win_bytes_backward,act_data_pkt_fwd,ActiveStd,ActiveMax,IdleStd,Label,Binary_class,attack_type
0,5435785,4,0,24,0,6,0.0,0,0.0,0.0,...,0,0,-1,3,0.0,952,0.0,DDoS,malicious,DOS
1,26,1,1,2,6,2,0.0,6,6.0,0.0,...,1,6,0,0,0.0,0,0.0,PortScan,malicious,PortScan
2,23475,1,1,70,152,70,0.0,152,152.0,0.0,...,1,152,-1,0,0.0,0,0.0,BENIGN,benign,benign
3,37553,2,0,0,0,0,0.0,0,0.0,0.0,...,0,0,-1,0,0.0,0,0.0,BENIGN,benign,benign
4,71220,4,4,172,384,43,0.0,96,96.0,0.0,...,4,384,-1,3,0.0,0,0.0,BENIGN,benign,benign


In [5]:
# Replace weird characters
label_names = df['Label'].unique()

import re
label_names = [re.sub("[^a-zA-Z ]+", "", l) for l in label_names]
label_names = [re.sub("[\s\s]", '_', l) for l in label_names]
label_names = [lab.replace("__", "_") for lab in label_names]
label_names, len(label_names)

# Replacing 'Label' column values with new readable values.
labels = df['Label'].unique()
for i in range(0,len(label_names)):
    df['Label'] = df['Label'].replace({labels[i] : label_names[i]})
    
df['Label'].unique()

array(['DDoS', 'PortScan', 'BENIGN', 'DoS_Hulk', 'DoS_slowloris',
       'SSHPatator', 'DoS_Slowhttptest', 'FTPPatator', 'DoS_GoldenEye',
       'Infiltration', 'Bot', 'Web_Attack_Brute_Force', 'Web_Attack_XSS',
       'Heartbleed', 'Web_Attack_Sql_Injection'], dtype=object)

In [6]:
df['Label'].value_counts().to_frame().sort_index(ascending=True)

,count
Label,
BENIGN,1048242
Bot,976
DDoS,64008
DoS_GoldenEye,5143
DoS_Hulk,86424
DoS_Slowhttptest,2614
DoS_slowloris,2692
FTPPatator,2966
Heartbleed,5


In [7]:
df

,FlowDuration,TotalFwdPackets,TotalBackwardPackets,TotalLengthofFwdPackets,TotalLengthofBwdPackets,FwdPacketLengthMin,FwdPacketLengthStd,BwdPacketLengthMax,BwdPacketLengthMean,BwdPacketLengthStd,...,SubflowBwdPackets,SubflowBwdBytes,Init_Win_bytes_backward,act_data_pkt_fwd,ActiveStd,ActiveMax,IdleStd,Label,Binary_class,attack_type
0,5435785,4,0,24,0,6,0.000000,0,0.0,0.000000,...,0,0,-1,3,0.0000,952,0.000000,DDoS,malicious,DOS
1,26,1,1,2,6,2,0.000000,6,6.0,0.000000,...,1,6,0,0,0.0000,0,0.000000,PortScan,malicious,PortScan
2,23475,1,1,70,152,70,0.000000,152,152.0,0.000000,...,1,152,-1,0,0.0000,0,0.000000,BENIGN,benign,benign
3,37553,2,0,0,0,0,0.000000,0,0.0,0.000000,...,0,0,-1,0,0.0000,0,0.000000,BENIGN,benign,benign
4,71220,4,4,172,384,43,0.000000,96,96.0,0.000000,...,4,384,-1,3,0.0000,0,0.000000,BENIGN,benign,benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1261172,67,1,1,2,6,2,0.000000,6,6.0,0.000000,...,1,6,0,0,0.0000,0,0.000000,PortScan,malicious,PortScan
1261173,53972647,10,10,432,2052,0,136.610395,2052,205.2,648.899376,...,10,2052,939,1,156701.8442,387543,2062.257986,BENIGN,benign,benign
1261174,30836,2,2,86,200,43,0.000000,100,100.0,0.000000,...,2,200,-1,1,0.0000,0,0.000000,BENIGN,benign,benign
1261175,60317,1,1,51,67,51,0.000000,67,67.0,0.000000,...,1,67,-1,0,0.0000,0,0.000000,BENIGN,benign,benign


### Sample dataframe by Benign into smaller size

In [8]:
# from sklearn.model_selection import train_test_split

# label_Benign_df1, label_Benign_df2 = train_test_split(df[df['Label'] == 'BENIGN'], test_size=0.02)
# # label_DDoS_df1, label_DDoS_df2 = train_test_split(df[df['Label'] == 'Bot'], test_size=0.05)
# # label_DDoS_df1, label_DDoS_df2 = train_test_split(df[df['Label'] == 'DDoS'], test_size=0.05)
# # label_DoS_GoldenEye_df1, label_DoS_GoldenEye_df2 = train_test_split(df[df['Label'] == 'DoS_GoldenEye'], test_size=0.05)
# # label_DoS_Hulk_df1, label_DoS_Hulk_df2 = train_test_split(df[df['Label'] == 'DoS_Hulk'], test_size=0.05)
# # label_DoS_Slowhttptest_df1, label_DoS_Slowhttptest_df2 = train_test_split(df[df['Label'] == 'DoS_Slowhttptest'], test_size=0.1)
# # label_DoS_slowloris_df1, label_DoS_slowloris_df2 = train_test_split(df[df['Label'] == 'DoS_slowloris'], test_size=0.05)
# # label_DoS_FTPPatator_df1, label_DoS_FTPPatator_df2 = train_test_split(df[df['Label'] == 'DoS_FTPPatator'], test_size=0.05)
# # label_PortScan_df1, label_PortScan_df2 = train_test_split(df[df['Label'] == 'PortScan'], test_size=0.01)
# # label_Web_Attack_Brute_Force_df1, label_Web_Attack_Brute_Force_df2 = train_test_split(df[df['Label'] == 'Web_Attack_Brute_Force'], test_size=0.05)
# # label_Web_Attack_XSS_df1, label_Web_Attack_XSS_df2 = train_test_split(df[df['Label'] == 'Web_Attack_XSS'], test_size=0.05)
# #label_Malicious_df1, label_Malicious_df2 = train_test_split(df[df['Label'] != 'BENIGN'], test_size=0.05)
# # label_Malicious_df = df[df['Label'] != 'BENIGN' and df['Label'] != 'DDoS' and df['Label'] != 'DoS_GoldenEye' and df['Label'] != 'DoS_Hulk' and df['Label'] != 'PortScan' and df['Label'] != 'Web_Attack_Brute_Force' and df['Label'] != 'Web_Attack_XSS'   ]
# label_Malicious_df1, label_Malicious_df2 = train_test_split(df[df['Label'] != 'BENIGN'], test_size=0.04)

# print("Benign set1: ", label_Benign_df1.shape)
# print("Benign set2: ", label_Benign_df2.shape)
# # print("DDoS set1: ", label_DDoS_df1.shape)
# # print("DDoS set2: ", label_DDoS_df2.shape)
# # print("DoS GoldenEye set1: ", label_DoS_GoldenEye_df1.shape)
# # print("DoS GoldenEye set2: ", label_DoS_GoldenEye_df2.shape)
# # print("DoS Hulk set1: ", label_DoS_Hulk_df1.shape)
# # print("DoS Hulk set2: ", label_DoS_Hulk_df2.shape)
# # print("PortScan set1: ", label_PortScan_df1.shape)
# # print("PortScan set2: ", label_PortScan_df2.shape)
# print("Malicious set1: ", label_Malicious_df1.shape)
# print("Malicious set2: ", label_Malicious_df2.shape)

# # df = pd.concat([label_Benign_df2, label_DDoS_df2, label_DoS_GoldenEye_df2, label_DoS_Hulk_df2, label_PortScan_df2, label_Malicious_df])
# df = pd.concat([label_Benign_df2, label_Malicious_df2])
# df

In [9]:
from sklearn.model_selection import train_test_split

label_Benign_df1, label_Benign_df2 = train_test_split(df[df['Binary_class'] == 'benign'], test_size=0.5)
label_Malicious_df1, label_Malicious_df2 = train_test_split(df[df['Binary_class'] == 'malicious'], test_size=0.5)
df = pd.concat([label_Benign_df2, label_Malicious_df2])
df


,FlowDuration,TotalFwdPackets,TotalBackwardPackets,TotalLengthofFwdPackets,TotalLengthofBwdPackets,FwdPacketLengthMin,FwdPacketLengthStd,BwdPacketLengthMax,BwdPacketLengthMean,BwdPacketLengthStd,...,SubflowBwdPackets,SubflowBwdBytes,Init_Win_bytes_backward,act_data_pkt_fwd,ActiveStd,ActiveMax,IdleStd,Label,Binary_class,attack_type
95235,30971,2,2,86,142,43,0.000000,71,71.000000,0.000000,...,2,142,-1,1,0.000000e+00,0,0.000000e+00,BENIGN,benign,benign
459558,5666302,2,1,12,0,6,0.000000,0,0.000000,0.000000,...,1,0,385,1,0.000000e+00,0,0.000000e+00,BENIGN,benign,benign
505317,51562962,105,189,8591,260537,0,265.861220,2920,1378.502646,549.562937,...,189,260537,45,104,2.773718e+06,5562552,3.530694e+02,BENIGN,benign,benign
1050673,6692,2,0,0,0,0,0.000000,0,0.000000,0.000000,...,0,0,-1,0,0.000000e+00,0,0.000000e+00,BENIGN,benign,benign
303648,119750498,22,21,1428,8169,0,149.763594,1737,389.000000,629.401462,...,21,8169,1015,21,8.093919e+05,2862684,1.064364e+06,BENIGN,benign,benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473890,381200,3,6,26,11607,0,10.263203,5840,1934.500000,2538.919278,...,6,11607,229,2,0.000000e+00,0,0.000000e+00,DDoS,malicious,DOS
24041,7816287,8,4,56,11601,0,5.656854,10135,2900.250000,4871.825830,...,4,11601,229,6,0.000000e+00,720322,0.000000e+00,DDoS,malicious,DOS
1088957,56,1,1,2,6,2,0.000000,6,6.000000,0.000000,...,1,6,0,0,0.000000e+00,0,0.000000e+00,PortScan,malicious,PortScan
11632,49,1,1,2,6,2,0.000000,6,6.000000,0.000000,...,1,6,0,0,0.000000e+00,0,0.000000e+00,PortScan,malicious,PortScan


### Drop NAN values

In [10]:
# Checking if there are any NULL values in the dataset.
df.isnull().values.any()

True

In [11]:
df.dropna(inplace=True)
df.isnull().values.any()

False

In [12]:
df

,FlowDuration,TotalFwdPackets,TotalBackwardPackets,TotalLengthofFwdPackets,TotalLengthofBwdPackets,FwdPacketLengthMin,FwdPacketLengthStd,BwdPacketLengthMax,BwdPacketLengthMean,BwdPacketLengthStd,...,SubflowBwdPackets,SubflowBwdBytes,Init_Win_bytes_backward,act_data_pkt_fwd,ActiveStd,ActiveMax,IdleStd,Label,Binary_class,attack_type
95235,30971,2,2,86,142,43,0.000000,71,71.000000,0.000000,...,2,142,-1,1,0.000000e+00,0,0.000000e+00,BENIGN,benign,benign
459558,5666302,2,1,12,0,6,0.000000,0,0.000000,0.000000,...,1,0,385,1,0.000000e+00,0,0.000000e+00,BENIGN,benign,benign
505317,51562962,105,189,8591,260537,0,265.861220,2920,1378.502646,549.562937,...,189,260537,45,104,2.773718e+06,5562552,3.530694e+02,BENIGN,benign,benign
1050673,6692,2,0,0,0,0,0.000000,0,0.000000,0.000000,...,0,0,-1,0,0.000000e+00,0,0.000000e+00,BENIGN,benign,benign
303648,119750498,22,21,1428,8169,0,149.763594,1737,389.000000,629.401462,...,21,8169,1015,21,8.093919e+05,2862684,1.064364e+06,BENIGN,benign,benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473890,381200,3,6,26,11607,0,10.263203,5840,1934.500000,2538.919278,...,6,11607,229,2,0.000000e+00,0,0.000000e+00,DDoS,malicious,DOS
24041,7816287,8,4,56,11601,0,5.656854,10135,2900.250000,4871.825830,...,4,11601,229,6,0.000000e+00,720322,0.000000e+00,DDoS,malicious,DOS
1088957,56,1,1,2,6,2,0.000000,6,6.000000,0.000000,...,1,6,0,0,0.000000e+00,0,0.000000e+00,PortScan,malicious,PortScan
11632,49,1,1,2,6,2,0.000000,6,6.000000,0.000000,...,1,6,0,0,0.000000e+00,0,0.000000e+00,PortScan,malicious,PortScan


### drop infinite values

In [13]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)
df

,FlowDuration,TotalFwdPackets,TotalBackwardPackets,TotalLengthofFwdPackets,TotalLengthofBwdPackets,FwdPacketLengthMin,FwdPacketLengthStd,BwdPacketLengthMax,BwdPacketLengthMean,BwdPacketLengthStd,...,SubflowBwdPackets,SubflowBwdBytes,Init_Win_bytes_backward,act_data_pkt_fwd,ActiveStd,ActiveMax,IdleStd,Label,Binary_class,attack_type
95235,30971,2,2,86,142,43,0.000000,71,71.000000,0.000000,...,2,142,-1,1,0.000000e+00,0,0.000000e+00,BENIGN,benign,benign
459558,5666302,2,1,12,0,6,0.000000,0,0.000000,0.000000,...,1,0,385,1,0.000000e+00,0,0.000000e+00,BENIGN,benign,benign
505317,51562962,105,189,8591,260537,0,265.861220,2920,1378.502646,549.562937,...,189,260537,45,104,2.773718e+06,5562552,3.530694e+02,BENIGN,benign,benign
1050673,6692,2,0,0,0,0,0.000000,0,0.000000,0.000000,...,0,0,-1,0,0.000000e+00,0,0.000000e+00,BENIGN,benign,benign
303648,119750498,22,21,1428,8169,0,149.763594,1737,389.000000,629.401462,...,21,8169,1015,21,8.093919e+05,2862684,1.064364e+06,BENIGN,benign,benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473890,381200,3,6,26,11607,0,10.263203,5840,1934.500000,2538.919278,...,6,11607,229,2,0.000000e+00,0,0.000000e+00,DDoS,malicious,DOS
24041,7816287,8,4,56,11601,0,5.656854,10135,2900.250000,4871.825830,...,4,11601,229,6,0.000000e+00,720322,0.000000e+00,DDoS,malicious,DOS
1088957,56,1,1,2,6,2,0.000000,6,6.000000,0.000000,...,1,6,0,0,0.000000e+00,0,0.000000e+00,PortScan,malicious,PortScan
11632,49,1,1,2,6,2,0.000000,6,6.000000,0.000000,...,1,6,0,0,0.000000e+00,0,0.000000e+00,PortScan,malicious,PortScan


### Save the dataframe

In [14]:
# save to csv
# df.to_csv('datasets/target_model/CICIDS2017/combined_data_0307_1.csv', index=False)

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import sklearn.metrics as m
import xgboost as xgb
from sklearn.model_selection import train_test_split



#Load Data
# df=pd.read_csv('datasets/target_model/CICIDS2017/combined_data_0307_1.csv')

X = df.drop(columns=['Label', 'Binary_class', 'attack_type'], axis=1)
y = df['Binary_class']

In [16]:
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()
df ['Binary_class'] = lb.fit_transform(df['Binary_class'])
labels = df['Binary_class']
labels.unique()

array([0, 1])

In [17]:
features = df.loc[:, df.columns != 'Binary_class']
features = df.drop(columns=['Label', 'attack_type'], axis=1)
df = features
df

,FlowDuration,TotalFwdPackets,TotalBackwardPackets,TotalLengthofFwdPackets,TotalLengthofBwdPackets,FwdPacketLengthMin,FwdPacketLengthStd,BwdPacketLengthMax,BwdPacketLengthMean,BwdPacketLengthStd,...,SubflowFwdPackets,SubflowFwdBytes,SubflowBwdPackets,SubflowBwdBytes,Init_Win_bytes_backward,act_data_pkt_fwd,ActiveStd,ActiveMax,IdleStd,Binary_class
95235,30971,2,2,86,142,43,0.000000,71,71.000000,0.000000,...,2,86,2,142,-1,1,0.000000e+00,0,0.000000e+00,0
459558,5666302,2,1,12,0,6,0.000000,0,0.000000,0.000000,...,2,12,1,0,385,1,0.000000e+00,0,0.000000e+00,0
505317,51562962,105,189,8591,260537,0,265.861220,2920,1378.502646,549.562937,...,105,8591,189,260537,45,104,2.773718e+06,5562552,3.530694e+02,0
1050673,6692,2,0,0,0,0,0.000000,0,0.000000,0.000000,...,2,0,0,0,-1,0,0.000000e+00,0,0.000000e+00,0
303648,119750498,22,21,1428,8169,0,149.763594,1737,389.000000,629.401462,...,22,1428,21,8169,1015,21,8.093919e+05,2862684,1.064364e+06,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473890,381200,3,6,26,11607,0,10.263203,5840,1934.500000,2538.919278,...,3,26,6,11607,229,2,0.000000e+00,0,0.000000e+00,1
24041,7816287,8,4,56,11601,0,5.656854,10135,2900.250000,4871.825830,...,8,56,4,11601,229,6,0.000000e+00,720322,0.000000e+00,1
1088957,56,1,1,2,6,2,0.000000,6,6.000000,0.000000,...,1,2,1,6,0,0,0.000000e+00,0,0.000000e+00,1
11632,49,1,1,2,6,2,0.000000,6,6.000000,0.000000,...,1,2,1,6,0,0,0.000000e+00,0,0.000000e+00,1


In [18]:
# save to csv
df.to_csv('../datasets/target_model/CICIDS2017/combined_data_new_0311_1.csv', index=False)

### SMOTE with imbalance data

In [19]:
# from imblearn.over_sampling import SMOTE
# y_rus_ =  df['Binary_class']
# X_rus_ =  df.drop(['Binary_class'],axis=1)

# sm = SMOTE(k_neighbors=3, random_state=42)
# X_sm, y_sm = sm.fit_resample(X_rus_, y_rus_)


In [20]:
# y_sm.value_counts()

In [21]:
# # draw the distribution of the data
# plt.figure(figsize=(5,5))
# y_sm.value_counts().plot(kind='pie', color=['skyblue', 'orange', 'red', 'green', 'blue', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan', 'black', 'yellow', 'lightgreen', 'lightblue'])

In [22]:
# # combine the data
# X_sm['Label'] = y_sm
# df_new = X_sm
# df_new

In [23]:
# # save to csv
# df_new.to_csv('datasets/target_model/CICIDS2017/combined_data_new_0307_1.csv', index=False)

### Train Test Split

In [24]:
# from sklearn.model_selection import KFold, cross_val_score, train_test_split



# # Initial train test split set
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# # Further split training set into training (70%) and validation (10%) sets
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, stratify=y_train)

# # Display the shapes of the resulting sets
# print("Training set shape:", X_train.shape)
# print("Validation set shape:", X_val.shape)
# print("Testing set shape:", X_test.shape)